# Pull code from https://github.com/flibustier/pokemon-tcg-pocket-database

In [10]:
import pandas as pd
import json

In [11]:
# Read JSON cards data from file
with open("/Users/72561/Documents/Personal/Trade_Pokemon/pokemon-tcg-pocket-database/dist/cards.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert JSON to DataFrame. 
# json_normalize automatically flattens nested objects like "label".
df = pd.json_normalize(data)

df.head()

,set,number,rarity,rarityCode,imageName,label.slug,label.eng
0,A1,1,Common,C,cPK_10_000010_00_FUSHIGIDANE_C.webp,bulbasaur,Bulbasaur
1,A1,2,Uncommon,U,cPK_10_000020_00_FUSHIGISOU_U.webp,ivysaur,Ivysaur
2,A1,3,Rare,R,cPK_10_000030_00_FUSHIGIBANA_R.webp,venusaur,Venusaur
3,A1,4,Double Rare,RR,cPK_10_000040_00_FUSHIGIBANAex_RR.webp,venusaur-ex,Venusaur ex
4,A1,5,Common,C,cPK_10_000050_00_CATERPIE_C.webp,caterpie,Caterpie


In [12]:
# import requests
# from io import BytesIO
# from PIL import Image
# from IPython.display import display

# # Example base URL: adjust as needed
# BASE_URL = "https://assets.pokemon-zone.com/game-assets/CardPreviews"

# def show_images_from_df(df):
#     """Display images inline for each row in the DataFrame."""
#     for idx, row in df.iterrows():
#         # Build the full image URL
#         image_url = f"{BASE_URL}/{row['imageName']}"
        
#         # Download the image
#         response = requests.get(image_url)
        
#         # If successful, display inline
#         if response.ok:
#             # Convert bytes into an image
#             image_data = BytesIO(response.content)
#             pil_image = Image.open(image_data)
#             display(pil_image)
#         else:
#             print(f"Failed to fetch image for {row['label.eng']} at {image_url}: HTTP {response.status_code}")


In [13]:
# show_images_from_df(df.head())

In [14]:
# Read JSON expansion data from file
with open("/Users/72561/Documents/Personal/Trade_Pokemon/pokemon-tcg-pocket-database/dist/sets.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert JSON to DataFrame. 
# json_normalize automatically flattens nested objects like "label".
df2 = pd.json_normalize(data)

df2.head()

,code,releaseDate,count,label.en
0,A1,2024-10-30,286.0,Genetic Apex
1,A1A,2024-12-17,85.0,Mythical Island
2,A2,2025-01-29,207.0,Space-Time Smackdown
3,A2A,2025-02-28,96.0,Triumphant Light
4,A2B,2025-03-27,111.0,Shining Revelry


In [15]:
#join two dfs on set - code
df_complete = df.merge(df2, left_on='set', right_on='code', suffixes=('', '_expansion'))
df_complete.head()

,set,number,rarity,rarityCode,imageName,label.slug,label.eng,code,releaseDate,count,label.en
0,A1,1,Common,C,cPK_10_000010_00_FUSHIGIDANE_C.webp,bulbasaur,Bulbasaur,A1,2024-10-30,286.0,Genetic Apex
1,A1,2,Uncommon,U,cPK_10_000020_00_FUSHIGISOU_U.webp,ivysaur,Ivysaur,A1,2024-10-30,286.0,Genetic Apex
2,A1,3,Rare,R,cPK_10_000030_00_FUSHIGIBANA_R.webp,venusaur,Venusaur,A1,2024-10-30,286.0,Genetic Apex
3,A1,4,Double Rare,RR,cPK_10_000040_00_FUSHIGIBANAex_RR.webp,venusaur-ex,Venusaur ex,A1,2024-10-30,286.0,Genetic Apex
4,A1,5,Common,C,cPK_10_000050_00_CATERPIE_C.webp,caterpie,Caterpie,A1,2024-10-30,286.0,Genetic Apex


In [16]:
df_final = df_complete[['label.en', 'label.eng', 'rarityCode', 'imageName']]
df_final.columns = ['Espansione', 'Nome', 'Rarità', 'Immagine']
df_final.head()

,Espansione,Nome,Rarità,Immagine
0,Genetic Apex,Bulbasaur,C,cPK_10_000010_00_FUSHIGIDANE_C.webp
1,Genetic Apex,Ivysaur,U,cPK_10_000020_00_FUSHIGISOU_U.webp
2,Genetic Apex,Venusaur,R,cPK_10_000030_00_FUSHIGIBANA_R.webp
3,Genetic Apex,Venusaur ex,RR,cPK_10_000040_00_FUSHIGIBANAex_RR.webp
4,Genetic Apex,Caterpie,C,cPK_10_000050_00_CATERPIE_C.webp


In [17]:
# Filter out cards that cannot be traded
df_final = df_final[df_final['Espansione'] != 'Promo A']
df_final = df_final[~df_final['Rarità'].isin(['SR', 'SAR', 'IM', 'UR', 'SR', 'S', 'SSR'])]

In [20]:
#print(df_final['Rarità'].unique())

mappa_simboli_rarità = {
    'C':  '♦',
    'U':  '♦♦',
    'R':  '♦♦♦',
    'RR': '♦♦♦♦',
    'AR': '★'
}

df_final['Rarità'] = df_final['Rarità'].map(mappa_simboli_rarità)

#print(df_final['Rarità'].unique())

['C' 'U' 'R' 'RR' 'AR']
['♦' '♦♦' '♦♦♦' '♦♦♦♦' '★']


In [21]:
df_final.to_csv('/Users/72561/Documents/Personal/Trade_Pokemon/pokemon-trade/static/files/Anagrafica_Pokemon.csv', index=False, encoding='utf-8-sig', sep=';')